# Chart Descriptions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andrewm4894/netdata-gpt-notebooks/blob/main/notebooks/chart_descriptions/chart_descriptions.ipynb)

In [132]:
# if running in colab uncomment the following line and run it to install the required packages
#!pip install python-dotenv netdata-pandas openai

In [133]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from netdata_pandas.data_cloud import get_data_cloud
import openai
import pprint as pp
from urllib.parse import urlparse
import requests
import json

# load tokens from .env file
load_dotenv()

NETDATA_API_TOKEN = os.getenv('NETDATA_API_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY

In [134]:
# inputs
space_id = 'ea93d7b8-0df6-45c0-b13d-1560996c89eb' 
room_id = 'd8a4e0c5-7c79-4145-900e-83a9f06fcb6a'
chart = None

In [135]:
def get_charts_cloud(space_id, room_id, api_token=None, base_url='https://app.netdata.cloud', node_ids=[]):
    """Get charts from netdata cloud api.
    """
    
    if api_token is None:
        api_token = os.getenv('NETDATA_API_TOKEN')
    
    base_url = 'https://app.netdata.cloud'
    url = f'{base_url}/api/v2/spaces/{space_id}/rooms/{room_id}/charts'
    headers = {'Accept': '*/*', 'Content-Type': 'application/json', 'Authorization': f'Bearer {api_token}'}
    data = {
        'filter': {
            'nodeIDs': node_ids,
        }
    }
    r = requests.post(url, headers=headers, data=json.dumps(data))
    
    if r.status_code != 200:
        
        print(f'Error: {r.status_code, r.text}')
        return None
    
    else:
            
        return r.json()['results']


charts = get_charts_cloud(space_id, room_id, api_token=NETDATA_API_TOKEN)

In [136]:
if chart == None:
    chart = np.random.choice(list(charts.keys()))

print(chart)

prometheus.aws-cluster-autoscaler.cluster_autoscaler_scale_down_in_cooldown


In [137]:
df = get_data_cloud(space_id, room_id, chart)
print(df.shape)
print(df.head())

dimensions = df.columns
print(dimensions)

(8, 1)
                     cluster_autoscaler_scale_down_in_cooldown
time                                                          
2023-04-01 20:53:57                                          0
2023-04-01 20:54:00                                          0
2023-04-01 20:54:03                                          0
2023-04-01 20:54:06                                          0
2023-04-01 20:54:09                                          0
Index(['cluster_autoscaler_scale_down_in_cooldown'], dtype='object')


In [138]:
chart_json = {}
chart_json['id'] = chart
chart_json['title'] = charts[chart]['title']
chart_json['dimensions'] = list(dimensions)
chart_json['units'] = charts[chart]['units']
chart_json['family'] = charts[chart]['family']
chart_json['context'] = charts[chart]['context']
chart_json['chart_type'] = charts[chart]['chartType']
chart_json['chart_labels'] = charts[chart]['chartLabels']
chart_json

{'id': 'prometheus.aws-cluster-autoscaler.cluster_autoscaler_scale_down_in_cooldown',
 'title': '[ALPHA] Whether or not the scale down is in cooldown. 1 if its, 0 otherwise',
 'dimensions': ['cluster_autoscaler_scale_down_in_cooldown'],
 'units': 'cooldown',
 'family': 'cluster_autoscaler',
 'context': 'prometheus.aws-cluster-autoscaler.cluster_autoscaler_scale_down_in_cooldown',
 'chart_type': 'line',
 'chart_labels': None}

In [139]:
prompt = f"""
You are an experienced SRE and sysadmin.

You are monitoring your infrastructure using Netdata Cloud.

You are documenting individual charts and their dimensions to help other users.

THe chart_json object is available to you and follows the format below:

```json
{{
    "id": "the chart id",
    "title": "the chart title",
    "dimensions": "a list of the dimensions",
    "units": "the units of the chart",
    "family": "the menu family of the chart",
    "context": "context of the chat. it follows structure like <type>.<name> where <type> also impacts where in the menu the chart appears and so can give hints of what it might relate to",
    "chart_type": "usually line or stacked or area - this is the type of chart"
}}
```

Here is the individual chart_json object describing the chart you are documenting:

```json
{chart_json}
```

Can you write short, useful and educational description of the chart and its dimensions? 

Please follow format like this:

```json
{{
    "chart_id": "{chart}",
    "chart_description": "<add description here>",
    "dimension_descriptions": [
        {{
            "<dimension name>": "<add dimension description here>",
            ...
        }}
}}
```
"""
print(prompt)


You are an experienced SRE and sysadmin.

You are monitoring your infrastructure using Netdata Cloud.

You are documenting individual charts and their dimensions to help other users.

THe chart_json object is available to you and follows the format below:

```json
{
    "id": "the chart id",
    "title": "the chart title",
    "dimensions": "a list of the dimensions",
    "units": "the units of the chart",
    "family": "the menu family of the chart",
    "context": "context of the chat. it follows structure like <type>.<name> where <type> also impacts where in the menu the chart appears and so can give hints of what it might relate to",
    "chart_type": "usually line or stacked or area - this is the type of chart"
}
```

Here is the individual chart_json object describing the chart you are documenting:

```json
{'id': 'prometheus.aws-cluster-autoscaler.cluster_autoscaler_scale_down_in_cooldown', 'title': '[ALPHA] Whether or not the scale down is in cooldown. 1 if its, 0 otherwise', 

In [140]:
# build messages list to pass to openai
messages=[
    {"role": "user", "content": prompt}
]

In [141]:
# call openai api
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
)

In [142]:
reply_content = completion.choices[0].message.content
#pp.pprint(reply_content)
print(reply_content)

{
    "chart_id": "prometheus.aws-cluster-autoscaler.cluster_autoscaler_scale_down_in_cooldown",
    "chart_description": "This chart shows whether or not the AWS cluster autoscaler is in a scale down cooldown period. It displays a value of 1 if it is in cooldown, and 0 if it is not.",
    "dimension_descriptions": [
        {
            "cluster_autoscaler_scale_down_in_cooldown": "This dimension represents whether or not the cluster autoscaler is in a scale down cooldown period. It is a binary value where 1 means it is in cooldown and 0 means it is not.",
        }
    ]
}
